# Chapter 1: Thinking Probabilistically

In [1]:
import matplotlib.pyplot as plt
import preliz as pz

## Exercise 2

Previously, we defined a Python function P to compute the probability of an event 
using the naive definition of probability. Generalize that function to compute the
probability of events when they are not all equally likely. Use this new function to
compute the probability of events 𝐴 and 𝐵 from the previous exercise. Hint: you can
pass a third argument with the probability of each event.

In [2]:
# Previous function
def P(S, A):
    if set(A).issubset(set(S)):
        return len(A) / len(S)
    else:
        return 0

In [3]:
# Exercise solution
def P(sample_space_probas: dict[str, float], events: set):
    if events.issubset(sample_space_probas.keys()):
        return sum([sample_space_probas[event] for event in events])
    else:
        return 0

sample_space_probas = {"s": 0.5, "b": 0.25, "c": 0.25}

In [4]:
events = {"s"}
P(sample_space_probas, events)

0.5

In [5]:
events = {"s", "b"}
P(sample_space_probas, events)

0.75

## Exercise 7

In [6]:
sample_sizes = [10, 100, 1000]